In [ ]:
import os
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn.model_selection import train_test_split
from sklearn import metrics
from sklearn.metrics import confusion_matrix
from sklearn.metrics import classification_report
from imblearn.over_sampling import SMOTE 
from sklearn import model_selection
from sklearn.linear_model import LogisticRegression
from sklearn.tree import DecisionTreeClassifier
from sklearn.ensemble import RandomForestClassifier
from sklearn.neighbors import KNeighborsClassifier
from sklearn.discriminant_analysis import LinearDiscriminantAnalysis
from sklearn.naive_bayes import GaussianNB
from sklearn.svm import SVC
from sklearn.neural_network import MLPClassifier
from sklearn.model_selection import GridSearchCV
import warnings
warnings.filterwarnings('ignore')
%matplotlib inline


data = pd.read_csv('/media/loop/Backup/experiment/New/Final_conventional.csv')

def class_l(x):

    if(x['MOS']>=1 and x['MOS']<2):
        return '1'  # poor
    if(x['MOS']>=2 and x['MOS']<3 ):
        return '2' # average
    if(x['MOS']>=3 and x['MOS']<5 ):
        return '3' # good
    
data=data.assign(mos=data.apply(class_l, axis=1)) 

#data.head(5)
data=data.dropna() #drop entire row due to missing value
#data['mos'].value_counts()
#sns.set(style="darkgrid")
#p = sns.countplot(data=data, x = 'mos')
#print(data.shape)
#data['mos'].value_counts()

feature=data.loc[:,data.columns.isin(['RTT','Throughput','Packets'])]#3
#X=X1.iloc[0:1725]
target=data.loc[:,data.columns.isin(['mos'])]#3
target.columns= ['mos']
#y=y1.iloc[0:1725]
#y=y1.iloc[0:1725]
#feature.shape
target.shape
smote=SMOTE(random_state=42) # resample all class same
X, y=smote.fit_resample(feature,target)
#print(X.shape, y.shape)

#sns.set(style="darkgrid")
#sns.countplot(data=y, x = 'mos')
#y['mos'].value_counts()

X_train, X_test,y_train, y_test = train_test_split( X, y, test_size=0.1, random_state=42)

###############################################################
# prepare configuration for cross validation test harness

seed = 42


# prepare models

param_grids=[]
##logistic
param_grids_0= {           
    'penalty':['l1', 'l2', 'elasticnet', 'none']
}
param_grids.append(param_grids_0)

##knn
param_grids_1 = {                      
    'n_neighbors': [3,5,7,10,25]
}
param_grids.append(param_grids_1)

## Decision Tree
param_grids_2 = { 
    'criterion':['gini', 'entropy'],
    'max_depth': [ 3, 10, 50, None],
    'min_samples_leaf': [3, 4, 5],
    'min_samples_split': [8, 10, 12],
    'class_weight': ['balanced',None]
}

param_grids.append(param_grids_2)
## NB
param_grids_3 = {   
    'priors':[None],
    'var_smoothing':[1e-09]
}

param_grids.append(param_grids_3)
 ## SVM
param_grids_4 = {  
     'gamma' :[0.001, 0.01, 0.1, 1],
     'kernel': ['linear', 'rbf', 'poly'],
     'degree' : [1,3,6],
     'C' : [0.001, 0.01, 0.1, 1, 10]

}
param_grids.append(param_grids_4)
 ### Random foresst
param_grids_5 = {      
    'bootstrap': [True],
    'max_depth': [ 3, 10, 50, 90, 100],
    'max_features': [3],
    'n_estimators': [100, 200,500],
    'min_samples_leaf': [3, 4, 5],
    'min_samples_split': [8, 10, 12],
    'class_weight': ['balanced',None]
}
param_grids.append(param_grids_5)
### MLP
param_grids_6 = {    
    'hidden_layer_sizes': [(50,50,50), (50,100,50), (100,)],
    'activation': ['identity', 'logistic', 'tanh', 'relu'],
    'solver': ['sgd', 'adam', 'lbfgs' ],
    'alpha': [0.0001, 0.05],
    'learning_rate': ['constant','adaptive'],
}
param_grids.append(param_grids_6)

models = []
models.append((LogisticRegression()))
models.append((KNeighborsClassifier()))
models.append((DecisionTreeClassifier()))
models.append((GaussianNB()))
models.append((SVC()))
models.append((RandomForestClassifier()))
models.append((MLPClassifier()))


best_estimator=[]


for i in range(7):
    #print(param_grids[i])
    #param_grids= param_grids_%d' % (i))
    grid_search = GridSearchCV(estimator = models[i], param_grid = param_grids[i],  cv =3, n_jobs = -1, verbose = 2, return_train_score=True)
    grid_search.fit(X_train, y_train.values.ravel())
    #print(grid_search.best_params_)
    
    best_estimator.append(grid_search.best_estimator_)
    

for j in range(7):
    print( best_estimator[j])
    print('\n')
    


